# QuASAr Benchmark Sweeps

This notebook provides reusable helpers to generate QuASAr benchmark circuits and run sweeps across circuit sizes or depths. Use the cells below to configure experiments, execute QuASAr, compare baseline simulators, and explore ablation studies.

## Setup
Import the unified circuit generator together with the QuASAr planning and execution stack.

In [ ]:
from __future__ import annotations

from contextlib import contextmanager
from typing import Any, Callable, Dict, Iterable, List, Optional

try:
    import pandas as pd  # type: ignore
except Exception:  # pragma: no cover - optional dependency
    pd = None

from IPython.display import display

from benchmarks.unified import generate_benchmark_circuit
from quasar.analyzer import AnalysisResult, analyze
from quasar.gate_metrics import circuit_metrics
from quasar.planner import PlannerConfig, plan
from quasar.simulation_engine import ExecutionConfig, execute_ssd
from quasar.SSD import PartitionNode, SSD
from quasar.baselines import run_single_baseline

## Helper utilities
The helper functions below orchestrate QuASAr runs, present tabular outputs, and configure ablation scenarios.

In [ ]:
BenchmarkRecord = Dict[str, Any]


def _to_dataframe(records: Iterable[BenchmarkRecord]):
    if pd is not None:
        return pd.DataFrame(list(records))
    return list(records)


@contextmanager
def disable_quick_path():
    import quasar.planner as planner_module

    original = planner_module._should_use_quick_path
    planner_module._should_use_quick_path = lambda ssd, cfg: False
    try:
        yield
    finally:
        planner_module._should_use_quick_path = original


def analyze_without_disjoint(circuit) -> AnalysisResult:
    metrics = circuit_metrics(circuit)
    ssd = SSD(meta={"total_qubits": circuit.num_qubits, "components": 1})
    node = PartitionNode(
        id=0,
        qubits=list(range(circuit.num_qubits)),
        circuit=circuit,
        metrics=dict(metrics),
    )
    ssd.add(node)
    return AnalysisResult(ssd=ssd, metrics_global=dict(metrics))


def run_quasar_pipeline(
    circuit,
    *,
    planner_overrides: Optional[Dict[str, Any]] = None,
    execution_overrides: Optional[Dict[str, Any]] = None,
    analysis_fn: Callable[[Any], AnalysisResult] = analyze,
    force_full_search: bool = False,
) -> Dict[str, Any]:
    analysis = analysis_fn(circuit)
    planner_cfg = PlannerConfig(**(planner_overrides or {}))

    exec_kwargs = dict(execution_overrides or {})
    exec_kwargs.setdefault("max_ram_gb", planner_cfg.max_ram_gb)
    exec_cfg = ExecutionConfig(**exec_kwargs)

    if force_full_search:
        with disable_quick_path():
            plan_result = plan(analysis.ssd, planner_cfg)
    else:
        plan_result = plan(analysis.ssd, planner_cfg)

    execution = execute_ssd(plan_result, exec_cfg)
    return {
        "analysis": analysis.metrics_global,
        "plan": {
            "estimated_cost": plan_result.estimated_cost,
            "num_partitions": len(plan_result.partitions),
            "decision_trace": list(plan_result.decision_trace),
        },
        "execution": execution,
    }


def summarize_run(outcome: Dict[str, Any]) -> Dict[str, Any]:
    analysis = outcome["analysis"]
    plan_meta = outcome["plan"]
    exec_meta = outcome["execution"]["meta"]
    return {
        "num_qubits": int(analysis.get("num_qubits", 0)),
        "num_gates": int(analysis.get("num_gates", 0)),
        "two_qubit_gates": int(analysis.get("two_qubit_gates", 0)),
        "estimated_cost": float(plan_meta["estimated_cost"]),
        "num_partitions": int(plan_meta["num_partitions"]),
        "decision_trace": plan_meta["decision_trace"],
        "exec_wall_s": float(exec_meta.get("wall_elapsed_s", 0.0)),
        "peak_rss_bytes": exec_meta.get("peak_rss_bytes"),
        "cache_hits": int(exec_meta.get("cache_hits", 0)),
        "cache_misses": int(exec_meta.get("cache_misses", 0)),
    }


def sweep_quasar(
    *,
    family: str,
    varying: str,
    values: Iterable[int],
    fixed_qubits: int,
    fixed_depth: int,
    circuit_kwargs: Optional[Dict[str, Any]] = None,
    planner_overrides: Optional[Dict[str, Any]] = None,
    execution_overrides: Optional[Dict[str, Any]] = None,
    analysis_fn: Callable[[Any], AnalysisResult] = analyze,
    force_full_search: bool = False,
) -> List[BenchmarkRecord]:
    records: List[BenchmarkRecord] = []
    for value in values:
        if varying == "qubits":
            num_qubits = int(value)
            depth = int(fixed_depth)
        elif varying == "depth":
            num_qubits = int(fixed_qubits)
            depth = int(value)
        else:
            raise ValueError("varying must be 'qubits' or 'depth'")
        circuit = generate_benchmark_circuit(
            num_qubits=num_qubits,
            depth=depth,
            family=family,
            **(circuit_kwargs or {}),
        )
        outcome = run_quasar_pipeline(
            circuit,
            planner_overrides=planner_overrides,
            execution_overrides=execution_overrides,
            analysis_fn=analysis_fn,
            force_full_search=force_full_search,
        )
        summary = summarize_run(outcome)
        summary.update({"family": family, "depth": depth})
        records.append(summary)
    return records


def sweep_baseline(
    *,
    backend: str,
    family: str,
    varying: str,
    values: Iterable[int],
    fixed_qubits: int,
    fixed_depth: int,
    circuit_kwargs: Optional[Dict[str, Any]] = None,
    baseline_kwargs: Optional[Dict[str, Any]] = None,
) -> List[BenchmarkRecord]:
    records: List[BenchmarkRecord] = []
    for value in values:
        if varying == "qubits":
            num_qubits = int(value)
            depth = int(fixed_depth)
        elif varying == "depth":
            num_qubits = int(fixed_qubits)
            depth = int(value)
        else:
            raise ValueError("varying must be 'qubits' or 'depth'")
        circuit = generate_benchmark_circuit(
            num_qubits=num_qubits,
            depth=depth,
            family=family,
            **(circuit_kwargs or {}),
        )
        result = run_single_baseline(
            circuit,
            backend,
            **(baseline_kwargs or {}),
        )
        payload = result.get("result", result)
        summary = {
            "backend": backend,
            "family": family,
            "num_qubits": num_qubits,
            "depth": depth,
            "ok": payload.get("ok"),
            "elapsed_s": payload.get("elapsed_s"),
            "wall_s_measured": payload.get("wall_s_measured"),
            "error": payload.get("error"),
        }
        records.append(summary)
    return records

## Default sweep configuration
Adjust the values in this cell to target specific circuit families, qubit counts, and depths.

In [ ]:
family = "mixed"
qubit_sweep = [6, 8, 10]
depth_sweep = [8, 12, 16]

# Circuit-generation options are passed to ``generate_benchmark_circuit``.
circuit_options = {"block_size": 4, "cutoff": 0.75, "tail_type": "sparse", "seed": 7}

# Planner overrides are forwarded to :class:`PlannerConfig`.
planner_options = {"max_ram_gb": 4.0, "prefer_dd": True}

# Baseline runner parameters, e.g. to cap RAM usage.
baseline_options = {"max_ram_gb": 4.0, "timeout_s": 30.0}

## QuASAr sweeps
Run QuASAr across the configured sweep dimensions.

In [ ]:
quasar_qubit_results = sweep_quasar(
    family=family,
    varying="qubits",
    values=qubit_sweep,
    fixed_qubits=qubit_sweep[0],
    fixed_depth=depth_sweep[0],
    circuit_kwargs=circuit_options,
    planner_overrides=planner_options,
)

quasar_depth_results = sweep_quasar(
    family=family,
    varying="depth",
    values=depth_sweep,
    fixed_qubits=qubit_sweep[0],
    fixed_depth=depth_sweep[0],
    circuit_kwargs=circuit_options,
    planner_overrides=planner_options,
)

display(_to_dataframe(quasar_qubit_results))
display(_to_dataframe(quasar_depth_results))

## Tableau baseline sweeps

In [ ]:
tableau_qubit_results = sweep_baseline(
    backend="tableau",
    family=family,
    varying="qubits",
    values=qubit_sweep,
    fixed_qubits=qubit_sweep[0],
    fixed_depth=depth_sweep[0],
    circuit_kwargs=circuit_options,
    baseline_kwargs=baseline_options,
)

display(_to_dataframe(tableau_qubit_results))

## Statevector baseline sweeps

In [ ]:
statevector_depth_results = sweep_baseline(
    backend="sv",
    family=family,
    varying="depth",
    values=depth_sweep,
    fixed_qubits=qubit_sweep[0],
    fixed_depth=depth_sweep[0],
    circuit_kwargs=circuit_options,
    baseline_kwargs=baseline_options,
)

display(_to_dataframe(statevector_depth_results))

## Decision-diagram baseline sweeps

In [ ]:
dd_qubit_results = sweep_baseline(
    backend="dd",
    family=family,
    varying="qubits",
    values=qubit_sweep,
    fixed_qubits=qubit_sweep[0],
    fixed_depth=depth_sweep[0],
    circuit_kwargs=circuit_options,
    baseline_kwargs=baseline_options,
)

display(_to_dataframe(dd_qubit_results))

## Ablation: disable disjoint parallelisation

In [ ]:
ablation_disjoint = sweep_quasar(
    family=family,
    varying="qubits",
    values=qubit_sweep,
    fixed_qubits=qubit_sweep[0],
    fixed_depth=depth_sweep[0],
    circuit_kwargs=circuit_options,
    planner_overrides=planner_options,
    analysis_fn=analyze_without_disjoint,
)

display(_to_dataframe(ablation_disjoint))

## Ablation: disable method-based partitioning

In [ ]:
ablation_method = sweep_quasar(
    family=family,
    varying="depth",
    values=depth_sweep,
    fixed_qubits=qubit_sweep[0],
    fixed_depth=depth_sweep[0],
    circuit_kwargs=circuit_options,
    planner_overrides={**planner_options, "hybrid_clifford_tail": False},
)

display(_to_dataframe(ablation_method))

## Ablation: disable quick-path method selection

In [ ]:
ablation_quick_path = sweep_quasar(
    family=family,
    varying="qubits",
    values=qubit_sweep,
    fixed_qubits=qubit_sweep[0],
    fixed_depth=depth_sweep[0],
    circuit_kwargs=circuit_options,
    planner_overrides=planner_options,
    force_full_search=True,
)

display(_to_dataframe(ablation_quick_path))